<a href="https://colab.research.google.com/github/awitz23/Seminararbeit_CS-GO_Spieltheorie/blob/main/Datei_zur_Er%C3%B6ffnungsstrategie/Strategiewahl_Code_mit_ID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installationen

In [ ]:
pip install Pillow==9.0.0

In [ ]:
#shell notwendig, da awpy golang benötigt

%%shell
add-apt-repository -y ppa:longsleep/golang-backports
apt -y update
apt -y install golang-go

pip install awpy


In [ ]:
!pip install nashpy
import nashpy as nash

import pandas as pd
import numpy as np
from awpy import *
from awpy.data import NAV_CSV
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
import plotly.graph_objects as go
import pandas as pd
import os
import ntpath
dir = '/content/drive/MyDrive/CS:GO_Dust2/2018_Dust2'

In [ ]:
#Initialisiere ein leeres Dataframe für spätere Nutzung
df_strategy=pd.DataFrame()

#Lege Pfad fest, aus dem die Dateien gelesen werden
dir = '/content/drive/MyDrive/CS:GO_Dust2/2018_Dust2'

#Strategiefindung - Parsing

In [ ]:
#Gehe den vorher definierten Pfad durch
for file in os.listdir(dir):
  #gehe für jedes Element, das auf .dem endet folgende Befehle durch
  if file.endswith(".dem"):
    #Lese alle Daten aus dem vorher definierten Pfad aus.
    p = DemoParser(demofile="/content/drive/MyDrive/CS:GO_Dust2/2018_Dust2/"+file, parse_rate=128)
    df = p.parse(return_type="df")
    id = ntpath.basename("/content/drive/MyDrive/CS:GO_Dust2/2018_Dust2/"+file)
    year = "18"

    #Strategiefindung für T-Seite
    #initialisiere einen Parameter, um die Runden in einer Datei durchzugehen
    i = 1

    Gameround = []
    Strategies = []
    Game_id =[]
    while i <= df["playerFrames"]["roundNum"].max():
      track3 = df["playerFrames"]
      track4 = track3[(track3['seconds'] > 10) & (track3['seconds'] <= 30)]
      track4 = track4.loc[track4["roundNum"]==i] 
      track4 = track4.loc[track4["side"]=="T"]
      
      #lese die Positionen der Navigation Mesh Datei ein
      t = NAV_CSV[NAV_CSV["mapName"] == "de_dust2"]


      area = []
      #für jede Reihe durchgehen und Position festlegen und in Liste einspeichern, für jede Sekunde für jeden Spieler
      for index, row in track4.iterrows():
         
        f = t[(t['northWestX'] > row['x']-40) & (t['northWestX'] < row['x']+40)]
        f = f[(f['northWestY'] > row['y']-40) & (f['northWestY'] < row['y']+40)]
      
        if f.empty:
          loc = "MISTAKE"
          area.append(loc)
        
        else:
          loc = f["areaName"].value_counts().idxmax()
          area.append(loc)

      #füge Liste mit Positionen zu DF hinzu, hierbei werden die Orte den Regionen zugeteilt
      track4["area"] = area 
      
      track4.loc[track4['area'] == "CTSpawn",'subgroup'] = "Mid"
      track4.loc[track4['area'] == "MidDoors",'subgroup'] = "Mid"
      track4.loc[track4['area'] == "TopofMid",'subgroup'] = "Mid"
      track4.loc[track4['area'] == "Middle",'subgroup'] = "Mid"
      track4.loc[track4['area'] == "Short",'subgroup'] = "Mid"

      track4.loc[track4['area'] == "ShortStairs",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "UnderA",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "ExtendedA",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "Catwalk",'subgroup'] = "BombA"

      track4.loc[track4['area'] == "BombsiteB",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "BDoors",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "Hole",'subgroup'] = "BombB"

      track4.loc[track4['area'] == "TSpawn",'subgroup'] = "TerroristSpawn"
      track4.loc[track4['area'] == "TRamp",'subgroup'] = "TerroristSpawn"

      track4.loc[track4['area'] == "OutsideTunnel",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "UpperTunnel",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "TunnelStairs",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "LowerTunnel",'subgroup'] = "BombB"

      track4.loc[track4['area'] == "LongA",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "LongDoors",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "OutsideLong",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "Pit",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "Ramp",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "Side",'subgroup'] = "BombA"

      track4.loc[track4['area'] == "BombsiteA",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "ARamp",'subgroup'] = "BombA"

      stra = []

      df2 = track4.groupby(['subgroup', 'name'])['subgroup'].count().unstack(fill_value=0)

      #erstelle eine Liste mit 5 Werten, pro Spieler ein Wert, in der gespeichert wird, in welcher Region der jeweilige Spieler die längste Zeit verbracht hat
      for column in df2:
        y = df2[{column}].idxmax()
        y = y.values.tolist()
        stra.append(y)
      
      j = 0
      
      #gehe Liste durch und finde Strategie für das jeweilige Team
      BombA = 0
      BombB = 0
      Mid = 0
      TerroristSpawn = 0

      for element in stra:
        if (element[j] == "BombA"):
          BombA = BombA+1
        elif (element[j] == "BombB"):
          BombB = BombB+1
        elif (element[j] == "Mid"):
          Mid = Mid+1
        elif (element[j] == "Terroristspawn"):
          TerroristSpawn = TerroristSpawn + 1 
      if BombA >= 4:
          strategy = "RushA"
          Strategies.append(strategy)
          strategy=0
          Gameround.append(i)
          Game_id.append(id+year)
      if BombB >=4:
          strategy = "RushB"
          Strategies.append(strategy)
          strategy=0
          Gameround.append(i)
          Game_id.append(id+year)
      if (BombA<4)&(BombB<4):
          strategy = "Split"
          Strategies.append(strategy)
          strategy=0
          Gameround.append(i)
          Game_id.append(id+year)

      i=i+1

    #Strategiefindung für CT-Seite
    i = 1
    Gameround = []
    CTStrategies = []
    Game_id =[]
    while i <= df["playerFrames"]["roundNum"].max():
      player= df["playerFrames"]
      track3 = player
      track4 = track3[(track3['seconds'] > 10) & (track3['seconds'] <= 30)]
      track4 = track4.loc[track4["roundNum"]==i] 
      track4 = track4.loc[track4["side"]=="CT"]
      
      t = NAV_CSV[NAV_CSV["mapName"] == "de_dust2"]


      area = []
      #für jede Reihe durchgehen und Position festlegen und in Liste einspeichern --> jede Sekunde für jeden Spieler
      for index, row in track4.iterrows():
        f = t[(t['northWestX'] > row['x']-80) & (t['northWestX'] < row['x']+80)]
        f = f[(f['northWestY'] > row['y']-80) & (f['northWestY'] < row['y']+80)]
      
        if f.empty:
          loc = "MISTAKE"
          area.append(loc)
        
        else:
          loc = f["areaName"].value_counts().idxmax()
          area.append(loc)

      #füge Liste mit Positionen zu DF hinzu
      track4["area"] = area 
      
      track4.loc[track4['area'] == "CTSpawn",'subgroup'] = "Mid"
      track4.loc[track4['area'] == "MidDoors",'subgroup'] = "Mid"
      track4.loc[track4['area'] == "TopofMid",'subgroup'] = "Mid"
      track4.loc[track4['area'] == "Middle",'subgroup'] = "Mid"
      track4.loc[track4['area'] == "Short",'subgroup'] = "Mid"

      track4.loc[track4['area'] == "ShortStairs",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "UnderA",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "ExtendedA",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "Catwalk",'subgroup'] = "BombA"

      track4.loc[track4['area'] == "BombsiteB",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "BDoors",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "Hole",'subgroup'] = "BombB"

      track4.loc[track4['area'] == "TSpawn",'subgroup'] = "TerroristSpawn"
      track4.loc[track4['area'] == "TRamp",'subgroup'] = "TerroristSpawn"

      track4.loc[track4['area'] == "OutsideTunnel",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "UpperTunnel",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "TunnelStairs",'subgroup'] = "BombB"
      track4.loc[track4['area'] == "LowerTunnel",'subgroup'] = "BombB"

      track4.loc[track4['area'] == "LongA",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "LongDoors",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "OutsideLong",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "Pit",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "Ramp",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "Side",'subgroup'] = "BombA"

      track4.loc[track4['area'] == "BombsiteA",'subgroup'] = "BombA"
      track4.loc[track4['area'] == "ARamp",'subgroup'] = "BombA"

      stra = []

      df2 = track4.groupby(['subgroup', 'name'])['subgroup'].count().unstack(fill_value=0)

      for column in df2:
        y = df2[{column}].idxmax()
        y = y.values.tolist()
        stra.append(y)
      
      j = 0
      

      BombA = 0
      BombB = 0
      Mid = 0
      TerroristSpawn = 0

      for element in stra:
        if (element[j] == "BombA"):
          BombA = BombA+1
        elif (element[j] == "BombB"):
          BombB = BombB+1
        elif (element[j] == "Mid"):
          Mid = Mid+1
        elif (element[j] == "Terroristspawn"):
          TerroristSpawn = TerroristSpawn + 1 
      if BombA >= 3:
          strategy = "FullA"
          CTStrategies.append(strategy)
          strategy=0
          Gameround.append(i)
          Game_id.append(id+year)
      if BombB >=3:
          strategy = "FullB"
          CTStrategies.append(strategy)
          strategy=0
          Gameround.append(i)
          Game_id.append(id+year)
      if (BombA<3)&(BombB<3):
          strategy = "Split"
          CTStrategies.append(strategy)
          strategy=0
          Gameround.append(i)
          Game_id.append(id+year)
      
      i=i+1
      
    Winner = df["rounds"]["winningSide"].tolist()


    df_strategy_temp = pd.DataFrame({'Round':Gameround, "T-strategy":Strategies, "CT-strategy":CTStrategies, "Winner":Winner, "ID":Game_id})
    df_strategy = df_strategy.append(df_strategy_temp)


#Strategiefindung - Datenaufbereitung

In [ ]:
#Aufteilung des großen DF in 9 kleine 
#Werte der T-Seite

pi_SS = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "T")]
pi_SA = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "T")]
pi_SB = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "T")]
pi_AS = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "T")]
pi_AA = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "T")]
pi_AB = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "T")]
pi_BS = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "T")]
pi_BA = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "T")]
pi_BB = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "T")]

In [ ]:
#Werte der CT-Seite 

CT_pi_SS = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "CT")]
CT_pi_SA = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "CT")]
CT_pi_SB = df_strategy[(df_strategy['T-strategy'] == "Split") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "CT")]
CT_pi_AS = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "CT")]
CT_pi_AA = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "CT")]
CT_pi_AB = df_strategy[(df_strategy['T-strategy'] == "RushA") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "CT")]
CT_pi_BS = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "Split") & (df_strategy['Winner'] == "CT")]
CT_pi_BA = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "FullA") & (df_strategy['Winner'] == "CT")]
CT_pi_BB = df_strategy[(df_strategy['T-strategy'] == "RushB") & (df_strategy['CT-strategy'] == "FullB") & (df_strategy['Winner'] == "CT")]

In [ ]:
#Gesamtzahl der Ereignisse 
Full_pi_SS = CT_pi_SS.shape[0] + pi_SS.shape[0]
Full_pi_SA = CT_pi_SA.shape[0] + pi_SA.shape[0]
Full_pi_SB = CT_pi_SB.shape[0] + pi_SB.shape[0]
Full_pi_AS = CT_pi_AS.shape[0] + pi_AS.shape[0]
Full_pi_AA = CT_pi_AA.shape[0] + pi_AA.shape[0]
Full_pi_AB = CT_pi_AB.shape[0] + pi_AB.shape[0]
Full_pi_BS = CT_pi_BS.shape[0] + pi_BS.shape[0]
Full_pi_BA = CT_pi_BA.shape[0] + pi_BA.shape[0]
Full_pi_BB = CT_pi_BB.shape[0] + pi_BB.shape[0]


#Strategiefindung - Tabellen

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [Full_pi_SS , Full_pi_AS , Full_pi_BS , Full_pi_SS + Full_pi_AS + Full_pi_BS ],
      [Full_pi_SA , Full_pi_AA , Full_pi_BA , Full_pi_SA + Full_pi_AA + Full_pi_BA ],
      [Full_pi_SB , Full_pi_AB , Full_pi_BB , Full_pi_SB + Full_pi_AB + Full_pi_BB ],
      [Full_pi_SS + Full_pi_SA + Full_pi_SB , Full_pi_AS + Full_pi_AA + Full_pi_AB ,
       Full_pi_BS + Full_pi_BA + Full_pi_BB , Full_pi_SS + Full_pi_AS + Full_pi_BS  +
       Full_pi_SA + Full_pi_AA + Full_pi_BA +
       Full_pi_SB + Full_pi_AB + Full_pi_BB ]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

#print("Gesamtzahl")
fig.show()

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [pi_SS.shape[0]/Full_pi_SS, pi_AS.shape[0]/Full_pi_AS, pi_BS.shape[0]/Full_pi_BS, pi_SS.shape[0]+ pi_AS.shape[0]+ pi_BS.shape[0]],
      [pi_SA.shape[0]/Full_pi_SA, pi_AA.shape[0]/Full_pi_AA, pi_BA.shape[0]/Full_pi_BA, pi_SA.shape[0]+ pi_AA.shape[0]+ pi_BA.shape[0]],
      [pi_SB.shape[0]/Full_pi_SB, pi_AB.shape[0]/Full_pi_AB, pi_BB.shape[0]/Full_pi_BB, pi_SB.shape[0]+ pi_AB.shape[0]+ pi_BB.shape[0]],
      [pi_SS.shape[0]+ pi_SA.shape[0]+ pi_SB.shape[0], pi_AS.shape[0]+ pi_AA.shape[0]+ pi_AB.shape[0],
       pi_BS.shape[0]+ pi_BA.shape[0]+ pi_BB.shape[0], pi_SS.shape[0]+ pi_AS.shape[0]+ pi_BS.shape[0] +
       pi_SA.shape[0]+ pi_AA.shape[0]+ pi_BA.shape[0]+
       pi_SB.shape[0]+ pi_AB.shape[0]+ pi_BB.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

#print("SiegWKEIT der T Seite")
fig.show()

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [CT_pi_SS.shape[0], CT_pi_AS.shape[0], CT_pi_BS.shape[0], CT_pi_SS.shape[0]+ CT_pi_AS.shape[0]+ CT_pi_BS.shape[0]],
      [CT_pi_SA.shape[0], CT_pi_AA.shape[0], CT_pi_BA.shape[0], CT_pi_SA.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_BA.shape[0]],
      [CT_pi_SB.shape[0], CT_pi_AB.shape[0], CT_pi_BB.shape[0], CT_pi_SB.shape[0]+ CT_pi_AB.shape[0]+ CT_pi_BB.shape[0]],
      [CT_pi_SS.shape[0]+ CT_pi_SA.shape[0]+ CT_pi_SB.shape[0], CT_pi_AS.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_AB.shape[0],
       CT_pi_BS.shape[0]+ CT_pi_BA.shape[0]+ CT_pi_BB.shape[0], CT_pi_SS.shape[0]+ CT_pi_AS.shape[0]+ CT_pi_BS.shape[0] +
       CT_pi_SA.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_BA.shape[0]+
       CT_pi_SB.shape[0]+ CT_pi_AB.shape[0]+ CT_pi_BB.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

#print("Siege der CT Seite")
fig.show()

In [ ]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b> </b>','<b>Split</b>','<b>FullA</b>','<b>FullB</b>','<b>Total</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['Split', 'RushA', 'RushB', '<b>TOTAL</b>'],
      [CT_pi_SS.shape[0]/Full_pi_SS, CT_pi_AS.shape[0]/Full_pi_AS, CT_pi_BS.shape[0]/Full_pi_BS, (CT_pi_SS.shape[0]+ CT_pi_AS.shape[0]+ CT_pi_BS.shape[0])/df_strategy.shape[0]],
      [CT_pi_SA.shape[0]/Full_pi_SA, CT_pi_AA.shape[0]/Full_pi_AA, CT_pi_BA.shape[0]/Full_pi_BA, (CT_pi_SA.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_BA.shape[0])/df_strategy.shape[0]],
      [CT_pi_SB.shape[0]/Full_pi_SB, CT_pi_AB.shape[0]/Full_pi_AB, CT_pi_BB.shape[0]/Full_pi_BB, (CT_pi_SB.shape[0]+ CT_pi_AB.shape[0]+ CT_pi_BB.shape[0])/df_strategy.shape[0]],
      [(CT_pi_SS.shape[0]+ CT_pi_SA.shape[0]+ CT_pi_SB.shape[0])/df_strategy.shape[0], (CT_pi_AS.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_AB.shape[0])/df_strategy.shape[0],
       (CT_pi_BS.shape[0]+ CT_pi_BA.shape[0]+ CT_pi_BB.shape[0])/df_strategy.shape[0], (CT_pi_SS.shape[0]+ CT_pi_AS.shape[0]+ CT_pi_BS.shape[0] +
       CT_pi_SA.shape[0]+ CT_pi_AA.shape[0]+ CT_pi_BA.shape[0]+
       CT_pi_SB.shape[0]+ CT_pi_AB.shape[0]+ CT_pi_BB.shape[0])/df_strategy.shape[0]]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

#print("SiegWKEIT der CT Seite")
fig.show()

#Strategiefindung - Nash

In [ ]:

A = np.array([[pi_SS.shape[0]/Full_pi_SS, pi_SA.shape[0]/Full_pi_SA, pi_SB.shape[0]/Full_pi_SB], 
              [pi_AS.shape[0]/Full_pi_AS, pi_AA.shape[0]/Full_pi_AA, pi_AB.shape[0]/Full_pi_AB],
              [pi_BS.shape[0]/Full_pi_BS, pi_BA.shape[0]/Full_pi_BA, pi_BB.shape[0]/Full_pi_BB]])

B = np.array([[CT_pi_SS.shape[0]/Full_pi_SS, CT_pi_SA.shape[0]/Full_pi_SA, CT_pi_SB.shape[0]/Full_pi_SB], 
              [CT_pi_AS.shape[0]/Full_pi_AS, CT_pi_AA.shape[0]/Full_pi_AA, CT_pi_AB.shape[0]/Full_pi_AB],
              [CT_pi_BS.shape[0]/Full_pi_BS, CT_pi_BA.shape[0]/Full_pi_BA, CT_pi_BB.shape[0]/Full_pi_BB]])

CSGO = nash.Game(A, B)
CSGO

In [ ]:
equilibria = CSGO.support_enumeration()
for eq in equilibria:
    print(eq)